# European Import-Export Geoanalysis

### Importing data: countries' position and import/export flows 
We start with downloading the data from the [Eurostat transport database](https://ec.europa.eu/eurostat/web/transport/data/database). We are interested in the positions of the countries, thus latitude and longitude, so that we can use this information to plot the countries of interest into a world map. 
We proceed with uploading the dataset containing the information of import and export activities and flows ('complete_dataset'). 
We update the position dataset with only the countries contained in the import/export dataset. 

In [23]:
import requests
import pandas as pd
import plotly.express as px

In [24]:
#Download the csv with countries and positions (latitude and longitude)
url = 'https://developers.google.com/public-data/docs/canonical/countries_csv'
html = requests.get(url).content
df_list = pd.read_html(html)
df_position = df_list[-1]

#import the complete dataset with import and export values
complete_dataset = pd.read_csv('official_dataset.csv')

#create list "l" of countries : from position dataset (df_position), select and append only the countries that are also present
# in the official_dataset (Only EU countries)
l = []
for i in list(df_position['name']):
    l.append(i in list(set(complete_dataset['REPORTER'])))

# update position dataframe (filtered in the step before)
df_position = df_position[l]
#export dataset
df_position.to_csv('position_countries.csv')


### Cleaning datasets
The next step consists in cleaning some countries' names with simpler and shorter ones for visualization purposes. We do this through a function.

In [31]:
def clean_df(df):
    '''
    Function that cleans the names of the counties in the dataset
    (e.g.  from 'Belgium (incl. Luxembourg LU -> 1998)' to 'Belgium')

    Parameters
    ----------
    df: pandas dataFrame that must be cleaned

    '''
    drop_df = df.loc[df['REPORTER'] == 'Czechia']
    df = df.drop(drop_df.index)
    df['REPORTER'] = df['REPORTER'].replace({'Belgium (incl. Luxembourg \'LU\' -> 1998)':'Belgium'})
    df['REPORTER'] = df['REPORTER'].replace({'Germany (incl. German Democratic Republic \'DD\' from 1991)':'Germany'})
    df['REPORTER'] = df['REPORTER'].replace({'Ireland (Eire)':'Ireland'})
    df['REPORTER'] = df['REPORTER'].replace({'Spain (incl. Canary Islands \'XB\' from 1997)':'Spain'})
    df['REPORTER'] = df['REPORTER'].replace({'France (incl. Saint Barth�lemy \'BL\' -> 2012; incl. French Guiana \'GF\', Guadeloupe \'GP\', Martinique \'MQ\', R�union \'RE\' from 1997; incl. Mayotte \'YT\' from 2014)':'France'})
    df['REPORTER'] = df['REPORTER'].replace({'Italy (incl. San Marino \'SM\' -> 1993)':'Italy'})
    return df

### Adding latitude and longitude information
We proceed with creating a function that appends latitute and longitude of each country name contained in the main dataset, in order to have one single csv file that can be used for all following analysis and plotting. 

In [27]:
def add_lat_lon(df, pos_countries):
    '''
    It adds the latitude and longitude to the main dataset, so that it is ready to
    be passed to plotly

    Parameters
    ----------
    df: main pandas dataFrame
    pos_countries: pandas DataFrame with latitude and longitude
    '''
    # create two dictionaries with the country and its relative position 
    d_lat = dict(zip(pos_countries['name'], pos_countries['latitude']))
    d_long = dict(zip(pos_countries['name'], pos_countries['longitude']))
    # empty lists that will become column of the dataset
    lat = []
    long = []
    for index, row in df.iterrows(): #there are only 500, that's way I iterate over the rows
        # append to each list the relative latitude and logitude of each country (with the right index)
        lat.append(d_lat[row['REPORTER']])
        long.append(d_long[row['REPORTER']])
    # create the new colomns 'Latitude' and 'Longitude' 
    df['Latitude'] = lat
    df['Longitude'] = long
    return df

### Merging countries' datasets 
In the following section we append all the information retrieved from the Eurostat database together, merging all countries' maritime import and export information in one single dataset.  

In [26]:
def merge_database(complete_df, already_merged = []):
    '''
    Return the joined dataset and a list of the names of the file already merged.

    If already_merged is not passed, it is considered empty.

    Parameters
    ----------
    complete_dataset: pandas DataFrame that you want to extend.
    '''
    # create an empty list where the dataset to merge will be added
    l = [] 
    for dir in  os.listdir(str(os.getcwd()) + '/dataset_project_DSA'):
        if dir[0:4] == 'data' and dir not in already_merged:
            # get the paths of the datasets that are not merged yet
            PATH = str(os.getcwd() + '/dataset_project_DSA/'+ dir +'/DS-1262527_1_Data.csv') 
            data = pd.read_csv(PATH)
            data = clean_df(data)
            # read the table with the position of each country 
            pos_countries = pd.read_csv(str(os.getcwd() + '/dataset_project_DSA/position_countries.csv'))
            # add the latitude and longitude for each dataset
            data = add_lat_lon(data, pos_countries)
            # append the modified dataset at the list l 
            l.append(data)
            # add ad the list of the already merged dataset the name of the directory
            already_merged.append(dir)
    # use pd.concat that merges all the datasets in a list (l) 
    datas = pd.concat(l)
    return datas, already_merged

if __name__ == '__main__':
    tup = merge_database([])
    tup[0].to_csv('./official_dataset.csv')


### Mapping Imports and Exports
As a final step, we plot the import and exports data.
Our final representation consists of two similar plots illustrating import and export flows respectively. The information can be inspected by cliking on the bubbles interactively.

Each bubble contains the following information: 
 - name of the partner
 - month and year of the specific flow 
 - value in euros of the flow
 - latitude of the country
 - longitude of the country
 
It is also possible to select single countries from the legend to further analyse the single relationship partner countries have with the European Union countries. 
A slider has been added to facilitate investigating how the flows changed over time. 

In [34]:
%%capture --no-display
official_df = pd.read_csv('./official_dataset.csv')
lat = official_df['Latitude'].unique()
lon = official_df['Longitude'].unique()

#subset dataset for import data
official_df_imp = official_df.loc[official_df['FLOW'] == 'IMPORT']
official_df_imp['Value'] = official_df_imp['Value'].str.replace(' ','').replace(':','0')
official_df_imp['Value'] = official_df_imp['Value'].astype(float)

#prepare dynamic map which shows evolution of imports over a two-year time
#the bubbles represent the size of imports of each EU countries from BRICS and USA
#it is possible to filter by partner and select the period to visualize

fig = px.scatter_geo(official_df_imp,
                    lat = 'Latitude',
                    lon = 'Longitude',
                    animation_frame = 'PERIOD',
                    size_max = 55,
                    hover_name = 'REPORTER',
                    size = 'Value',
                    color = 'PARTNER',
                    title="IMPORTS of EU countries from BRICS and USA")

fig.update_geos(fitbounds="locations", showcountries = True)
#plot map
fig.show()


#plot dynamic map which shows evolution of exports over a two-year time
#the bubbles represent the size of exports from each EU countries to BRICS and USA
#it is possible to filter by partner and select the period to visualize

official_df_exp = official_df.loc[official_df['FLOW'] == 'EXPORT']
official_df_exp['Value'] = official_df_exp['Value'].str.replace(' ','').replace(':','0')
official_df_exp['Value'] = official_df_exp['Value'].astype(float)


fig = px.scatter_geo(official_df_exp,
                    lat = 'Latitude',
                    lon = 'Longitude',
                    animation_frame = 'PERIOD',
                    size_max = 55,
                    hover_name = 'REPORTER',
                    size = 'Value',
                    color = 'PARTNER',
                    title="EXPORTS from EU countries to BRICS and USA")

fig.update_geos(fitbounds="locations", showcountries = True)
#plot map
fig.show()



### Conclusion 
The data observed are interesting, especially when looking at the change over time. It is possible to notice a significant spike in flows when going from December 2020 to January 2021.
We observe indeed that in December 2020 there is a significant increase in both export and imports.
Concerning imports, the main countries from which the EU imports seem to be China and the US, with Russia being present to some extent in some of the border countries as Poland and the Balcan States. 
When looking at exports, it is evident that the United States are the main exporte partner of EU states, with no similar competitor. 

### Further considerations 
The value of this project can be easily extended by using the same analysis with different partner countries or observing different timeframes. 